# Run Galfit on 3DHST galaxies
- 11/09/2020: Updated to use galfit_helpers.py module

### Load modules

In [1]:
from __future__ import division
import sys
sys.path.insert(0,'/data/emiln/XLSSC122_GalPops/Analysis/Modules')
from GalfitPyWrap import galfitwrap as gf
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import glob
from scipy import ndimage
from astropy import units as u
from astropy import wcs
from astropy.coordinates import SkyCoord
from astropy.io import fits, ascii
from astropy.table import Table, hstack, join
import pickle
import fnmatch
import multiprocessing as mp
%matplotlib inline
%load_ext autoreload
%autoreload 2

/data/emiln/miniconda2/lib/python2.7/site-packages/pyfits/__init__.py:22: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  PyFITSDeprecationWarning)  # noqa


In [2]:
import galfit_helpers as gfh

### Load 3DHST galaxy catalog

In [12]:
DHST_cat_root = '/data/emiln/XLSSC122_GalPops/Data/3DHST/Products/catalogs/'
DHST_cat_filename = DHST_cat_root+'cosmos_3dhst.v4.1.cats/Catalog/cosmos_3dhst.v4.1.cat'
# full_df = gfh.load_3DHST_galaxy_catalog(DHST_cat_filename, mag='F140W',magthresh=None, overwrite=False, verbose=True,goodfit=False)
full_df = gfh.load_3DHST_galaxy_catalog(DHST_cat_filename, mag='F140W',magthresh=30, overwrite=False, verbose=True,goodfit=False)
z2_filtered_df = gfh.load_3DHST_galaxy_catalog(DHST_cat_filename,  mag='F140W', magthresh=24, z=2.00, z_thresh=0.1, overwrite=False, verbose=True, goodfit=True) 

Checking if catalog exists at /data/emiln/XLSSC122_GalPops/Data/3DHST/Products/catalogs/magthresh30.csv
Catalog already exists, returning DataFrame
Checking if catalog exists at /data/emiln/XLSSC122_GalPops/Data/3DHST/Products/catalogs/magthresh24_z2.0_zthresh0.1.csv
Catalog already exists, returning DataFrame


### Set Galfit parameters

In [23]:
# fit_df = filtered_df[0:4]
fit_df = z2_filtered_df[0:4]
df_name = 'TEST_modular_galfit'
psf_root = '/data/emiln/XLSSC122_GalPops/Data/3DHST/Products/PSF/'
psf_file = psf_root+'cosmos_3dhst.v4.0.F140W_psf.fits'
PSFf = 1 
convbox='40 40'
image_width = 400 # 200 for mag<23 objects, 400 for z2 objects
timeout=2*60
verbose=True
width=10
HLRwidth=10 # Cutout width = 2*HLwidth
PA_INIT = 45
AR_INIT = 0.5
# ZP = 26.465 # https://iopscience.iop.org/article/10.1088/0067-0049/214/2/24/pdf page 13
ZP = 25
# page 20 --> All cataloged fluxes are normalized to AB zero point = 25
MAG_INIT = 21
fitMagnitude = True
neighbourMagThresh=2
DYNMAG = True
sky='Default'
sky_INIT=0.00001
sigma_file = '/sigma_rms_meanexp_cps.fits'
data_file = '/data_cps.fits'
constraint_file = 'galfit_constr.txt' # Matches van der wel 2012 constraints
# constr_file = 'none'
badmask='none'
save_name = None

In [24]:
params = {
    'survey':'3DHST',
    'fit_df':fit_df, # Dataframe with objects to be fit
    'full_df':full_df, # Unfiltered source catalog used for fitting neighbours
    'width':width, # Fitting region width in pixels
    'HLRwidth':HLRwidth, # Fitting region width in # of HLR
    'sigma_file':sigma_file, # Filename of sigma maps for sources
    'data_file':data_file, # Filename for data cutouts for each source
    'PSF_file':psf_file, # File_name of PSF to be used
    'usePSF':True, # Use PSF in fitting?
    'timeout':timeout, # Max runtime per object in seconds
    'PSFf':PSFf, # Fine sampling factor
    'verbose':verbose, # Verbose mode
    'PA_INIT':PA_INIT, # Initial position angle
    'AR_INIT':AR_INIT, # Initial axis ratio
    'MAG_INIT':MAG_INIT, # Initial magnitude
    'convbox':convbox, # Region to convolve PSF with in pixels (i.e. '100 100')
    'ZP':ZP, # Zeropoint 
    'constraint_file':constraint_file, # Galfit constraint filename
    'image_width':image_width, # Size of data+sigma images being used (200 for COSMOS cutouts)
    'useDYNMAG':DYNMAG, # Initialize magnitudes from catalog?
    'badmask':badmask, # filename for bad pixel mask
    'fitMagnitude':fitMagnitude, # Fit magnitudes?
    'sky':sky, # Sky fitting mode for galfit (i.e. 'default')
    'sky_INIT':sky_INIT, # Initial sky level
    'neighbourMagThresh':neighbourMagThresh, # Additional magnitude threshhold to fit neighbours (i.e. 3 -> only neighbours with mag < source_mag+3 are fit)
    'df_name': df_name, # Descriptive name of catalog being fit
    'save_name':save_name # Filename to save results to, overrides default
}

In [25]:
# *** NEED TO UPDATE HARDCODED MAGS IN RUN GALFIT FOR 3DHST ***
new_df, savename = gfh.run_galfit_parallel(params,**params) # Feeds in unwrapped params as well as a copy to be passed as kwargs to sub functions

   : [400.00, 400.00]  -3.88e-03  [0.00e+00]  [0.00e+00]
COUNTDOWN = 7 

Iteration : 17    Chi2nu: 1.683e-01     dChi2/Chi2: 4.16e-08    alamda: 1e-01     
 sersic    : (  399.93,   399.33)   23.26      3.61    2.17    0.45   -20.50
 sersic    : (  427.95,   368.18)   24.55     13.82    0.38   *0.01*   47.99
 sersic    : (  425.71,   417.16)   24.30      2.53    3.73   *0.01*  -24.19
 sky       : [400.00, 400.00]  -3.88e-03  [0.00e+00]  [0.00e+00]
COUNTDOWN = 6 

Iteration : 18    Chi2nu: 1.680e-01     dChi2/Chi2: -1.91e-03   alamda: 1e-02     
 sersic    : (  399.94,   399.33)   23.25      3.82    2.56    0.41   -23.02
 sersic    : (  427.99,   368.21)   24.59     15.82    0.24   *0.01*   47.93
 sersic    : (  425.76,   417.05)   24.14      3.29    2.10   *0.01*  -24.02
 sky       : [400.00, 400.00]  -3.91e-03  [0.00e+00]  [0.00e+00]
COUNTDOWN = 82 

Iteration : 19    Chi2nu: 1.680e-01     dChi2/Chi2: 7.69e-09    alamda: 1e-01     
 sersic    : (  399.94,   399.33)   23.25      3.82  

### Validate results

In [22]:
new_df

,ID,ra,dec,re,ar,n,mag,sky,chi2nu,ErrorValue
0,691,150.104736,2.187411,-99.0,-99.0,-99.0,-99.0,-99.0,1000,1000
1,2140,150.147583,2.201060,-99.0,-99.0,-99.0,-99.0,-99.0,1000,1000
2,2331,150.138916,2.202856,-99.0,-99.0,-99.0,-99.0,-99.0,1000,1000
3,2712,150.138916,2.206575,-99.0,-99.0,-99.0,-99.0,-99.0,1000,1000


In [ ]:
# Plot fits by ID
for ID in fit_df['ID']:
    gfh.plot_by_ID(ID,save_name=savename,survey='3DHST')